# 2. Versuch für Sequenze prediction Transformer

Ich versuche jetzt das objective aus dem Guide: https://towardsdatascience.com/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1

mit der referenz aus dem offiziellen pytorch Tutorial: https://pytorch.org/tutorials/beginner/translation_transformer.html

Die Schritte sind:

1. Daten Generieren/Vorbereiten -> Auf die Modellhyperparameter achten -> müssen zu Daten passen
    - Daten Sind sentences(sequenzen) zu bestimmten längen (z.B. 8), und sind in Batches vorliegend
    - Daten Brauchen Start of Stream/ End of Stream tokens oder beide
2. Modell Definieren
    - Positional encoding selber definieren (Vorlage nehmen)
    - Die Transformerstruktur definieren (Hier viele Bauteile von Pytorch verwenden)
    - Das Masking-zeug selber definieren
    - Das Padding zeug evtl selber definieren
3. Training/Validation definieren
    - Modell initialisieren
    - Optimizer Festlegen
    - Kostenfunktion festlegen
    - Trainingsfunktion festlegen (Muss nicht alles in der Trainingsfunktion direkt passieren, aber es muss passieren)
        - Es muss beim Training diese Verschiebung der Tokens passieren, dass der nächste output für eine Sequenz ausgegeben wird
        - Target-tensor wird während der Prediction ans Modell gegeben
        - Target-Maske muss Generiert werden 
        - Padding maske muss evtl auch generiert werden
    - Validationfunktion festlegen
        - Ist das gleiche wie im Training, nur werden hier keine Gradienten geupdated oder gelesen
4. Training/Validation ausführen
5. Inferenz ausführen

## 0. Imports

In [133]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

import math
import numpy as np

import random

Gpu nutzen wenn möglich

In [134]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1. Daten Generieren/Vorbereiten

Wir generieren Sequenzes für die Sequenze prediction. Sequenzen sollen so aussehen:

- 1, 1, 1, 1, 1, 1, 1, 1 → 1, 1, 1, 1, 1, 1, 1, 1
- 0, 0, 0, 0, 0, 0, 0, 0 → 0, 0, 0, 0, 0, 0, 0, 0
- 1, 0, 1, 0, 1, 0, 1, 0 → 1, 0, 1, 0, 1, 0, 1, 0
- 0, 1, 0, 1, 0, 1, 0, 1 → 0, 1, 0, 1, 0, 1, 0, 1



- Alle Sequenzen haben die länge 8 -> kein Padding nötig
- Sequenzen werden Zufällig in Batches der Größe 16 eingeteilt
- Hier Werden auch die Start of Stream und End of stream Tokens Vorne und hinten and Die Sequenzen gehängt

**Obacht: Unsere Daten sind anders angeordnet als die Daten aus dem Übersetzer Tutorial**

In [135]:
def generate_random_data(n):
    SOS_token = np.array([2])
    EOS_token = np.array([3])
    length = 8

    data = []

    # 1,1,1,1,1,1 -> 1,1,1,1,1
    for i in range(n // 3):
        X = np.concatenate((SOS_token, np.ones(length), EOS_token))
        y = np.concatenate((SOS_token, np.ones(length), EOS_token))
        data.append([X, y])

    # 0,0,0,0 -> 0,0,0,0
    for i in range(n // 3):
        X = np.concatenate((SOS_token, np.zeros(length), EOS_token))
        y = np.concatenate((SOS_token, np.zeros(length), EOS_token))
        data.append([X, y])

    # 1,0,1,0 -> 1,0,1,0,1
    for i in range(n // 3):
        X = np.zeros(length)
        start = random.randint(0, 1)

        X[start::2] = 1

        y = np.zeros(length)
        if X[-1] == 0:
            y[::2] = 1
        else:
            y[1::2] = 1

        X = np.concatenate((SOS_token, X, EOS_token))
        y = np.concatenate((SOS_token, y, EOS_token))

        data.append([X, y])

    np.random.shuffle(data)

    return data


def batchify_data(data, batch_size=16, padding=False, padding_token=-1):
    batches = []
    for idx in range(0, len(data), batch_size):
        # We make sure we dont get the last bit if its not batch_size size
        if idx + batch_size < len(data):
            # Here you would need to get the max length of the batch,
            # and normalize the length with the PAD token.
            if padding:
                max_batch_length = 0

                # Get longest sentence in batch
                for seq in data[idx : idx + batch_size]:
                    if len(seq) > max_batch_length:
                        max_batch_length = len(seq)

                # Append X padding tokens until it reaches the max length
                for seq_idx in range(batch_size):
                    remaining_length = max_batch_length - len(data[idx + seq_idx])
                    data[idx + seq_idx] += [padding_token] * remaining_length

            batches.append(np.array(data[idx : idx + batch_size]).astype(np.int64))

    print(f"{len(batches)} batches of size {batch_size}")

    return batches


train_data = generate_random_data(9000)
val_data = generate_random_data(3000)

train_dataloader = batchify_data(train_data)
val_dataloader = batchify_data(val_data)

### Daten Anschauen

train data:
- Liste der Länge 9000
    - Jedes Element ist weiter liste mit 2 Elementen
        - Das eine Element ist Numpy-array der länge 10 -> 8 Tokens und die Eos und SOS tokens -> Source sequence
        - Das andere ist Gleiches Array mit Target sequence

In [136]:
print("Anzahl der Elemente in Trainingsdaten:", len(train_data))

print("Anzahl der Elemente in einem Trainingssatz", len(train_data[0]))

print("Dimension eines der Seqeunzen in einem Trainingssatz", train_data[0][0].shape)

train_dataloader sind die Trainingsdaten gebatched in zufällige batches von 16 unterteilt

- liste der länge 562 (9000 / 16 = 562.5 -> der letze "halbe" batch wird nicht mehr genommen)
    - Jedes Element ist ein numpy-array mit den Dimensionen (16, 2, 10)
        - die Jeweiligen 16 Batches sind die 2 sequenzen die zusammengehören (2, 10)
        - Die jeweiligen zusammengehörenden Sequenzen sind die insgesamt 10 Tokens

In [137]:
print("Anzahl der Batches:", len(train_dataloader))

print("Dimensionen in einem Batch:", train_dataloader[0].shape)

## 2. Modell Definieren

### 2.1 Das Positional Encoding definieren

Wird einfach die Sinus-cosinus Formel aus dem Attention is all you need paper in code gepackt

Fügt informationen über die Word-order hinzu.

In [138]:
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens in the sequence.
        The positional encodings have the same dimension as the embeddings, so that the two can be summed.
        Here, we use sine and cosine functions of different frequencies.
    .. math:
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        # Info
        self.dropout = nn.Dropout(p=dropout)
        
        # Encoding - From formula -> This is basically applying the formula for Positional encoding (The one with Sinus and Cosinus)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # Baically a positions list 0, 1, 2, 3, 4, 5, ...
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # 1000^(2i/dim_model)
        
        # # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        
        #  # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pe[:, 1::2] = torch.cos(position * div_term)
        
         # Saving buffer (same as parameter without gradients needed)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """
        
        # Residual connection + pos encoding
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

### 2.2 Die Transformerstruktur definieren

ich versuche die Definition aus dem Pytorch tutorial Für den Guide anzupassen.

Beachte: Die Definition der Layer ist hier nicht in der Schönen Gleichen Reihenfolge wie im Guide

In [139]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        
        # Hier werden glaube ich die Layer definiert. Ist Im guide glaube ich in anderer Reihenfolge -> hab sie jetzt in die gleiche Reihenfolge wie im guide gepackt
        
        ## Layers des Gesamten Modells
        
        # Positional Encoding zur Hinzufügung von Positionsinformationen zu den Token-Einbettungen
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)
        
        # Token-Einbettung für Quell- und Zielvokabular
        # I use a nn.Embedding instead of the self defined TokenEmbedding
        self.src_tok_emb = nn.Embedding(src_vocab_size, emb_size) 
        self.tgt_tok_emb = nn.Embedding(tgt_vocab_size, emb_size)
        
        # Initialisierung des nn.Transformer Moduls mit den gegebenen Hyperparametern
        self.transformer = nn.Transformer(d_model=emb_size,
                                          nhead=nhead,
                                          num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers,
                                          dim_feedforward=dim_feedforward,
                                          dropout=dropout)
        
        # Linearer Layer zur Projektion der Ausgabedimensionen auf die Zielvokabulargröße
        # Generator ist also glaube ich die Outputlayer, die die Ausgabe in die Wahrscheinlichkeiten für die einzelnen Tokens übersetzt
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        
        

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        
        
        # Einbettung und Positional Encoding für die Quellsequenz
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        
        # Einbettung und Positional Encoding für die Zielsequenz
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        
        # Durchführen der Transformationsoperation
        # src_emb und tgt_emb sind die eingebetteten Sequenzen mit Positionsinformationen
        # src_mask und tgt_mask sind die Masken, die verhindern, dass zukünftige Tokens betrachtet werden
        # src_padding_mask, tgt_padding_mask und memory_key_padding_mask sind die Masken für Padding-Tokens
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        
        # Projektion der Ausgabe auf die Zielvokabulargröße
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        # Einbettung und Positional Encoding für die Quellsequenz
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        
        # Encoder-Durchlauf mit Quellsequenz und Maske
        return self.transformer.encoder(src_emb, src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        # Einbettung und Positional Encoding für die Zielsequenz
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(tgt))
        
        # Decoder-Durchlauf mit Zielsequenz, Gedächtnis und Maske
        return self.transformer.decoder(tgt_emb, memory, tgt_mask)

### 2.3 Maske definieren

- Wir brauchen ein Maske für die masked attention layer(s), sodass das modell beim Training nicht Die zukunft sehen kann (Die zukünftigen Tokens in der Sequenz sieht)
- Wir Brauche evtl. eine Maske, die Anzeigt, bei welchen Tokens es sich um padding-tokens handelt

Da bei der Makendefinition die Special Tokens wichtig sind, müssen Die special Tokens Genau gekenzeichnet werden. -> Häng von den Daten ab, da dort meißt die special tokens Definiert werden.

z.B. bei dem Übersetzungs-pytorch tutorial:
- UNK_IDX -> wahrscheinlich für Unknown_Token: This index is returned when the token is not found
- PAD_IDX -> Padding_index: Der Token, der Für fehlende Daten genutz wird, sodass die Dimensionen passen
- BOS_IDX -> Beginning of Stream Token -> Zeigt an, dass jetzt die Sequenz startet.
- EOS_IDX -> End of Stream Token -> Zeigt an, dass stream jetzt zuende ist.

```
# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)
```

Bei Uns Wurden in der Datengenerierung für die Tokens folgendes gemacht:

```
SOS_token = np.array([2])
EOS_token = np.array([3])
```
Also sollte der SOS (Start of stream)/ BOS (Beginning of stream) token ein numpy array mit einer 2 sein
Der EOS token sollte ein numpy array mit einer 3 sein

Also müssen wir unsere Tokens auch definieren:


In [140]:
UNK_IDX = 5 #Brauchen wir glaube ich nicht
PAD_IDX = 4 #Brauchen wir auch nicht, weil die sequenzen alle genau richtig lang sind
BOS_IDX = 2 # So gesetzt wie im Guide beispiel mit den sequenzen
EOS_IDX = 3 #auch gesetz wie im guide (hoffe ich)

Jetzt die Funktionen für die masken selbst definieren: (Vorlage aus dem übersetzter Tutorial)

In [141]:
# Generiert eine Obere Dreiecksmatrix, die in eine Untere Dreiecksmatrix transponiert wird
# -> Deckt also nach und nach ein Token auf
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    
    # EX for size=5:
    # [[0., -inf, -inf, -inf, -inf],
    #  [0.,   0., -inf, -inf, -inf],
    #  [0.,   0.,   0., -inf, -inf],
    #  [0.,   0.,   0.,   0., -inf],
    #  [0.,   0.,   0.,   0.,   0.]]
    
    return mask

# Die Funktion create_mask erstellt sowohl Quell- als auch Ziel-Pad-Masken, indem sie prüft, ob Elemente in der Quell- und Zielsequenz gleich dem Pad-Token sind. Diese Masken werden transponiert, um die richtige Dimension zu erhalten.
def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

## 3 Training/Validation definieren

### 3.1 Modell initialisieren

Jetzt das Modell initialisieren mit den zu den Daten passenden Hyperparametern

Hyperparameter:

In [142]:
SRC_VOCAB_SIZE = 4 # Ist glaube ich das num_tokens aus dem Guide (Also wie viele Verschiedene Tokens es insgesamt gibt
TGT_VOCAB_SIZE = 4 # Auch 4, da die eingabe und zielsequenz die Gleichen möglichkeiten für Tokens haben
EMB_SIZE = 8 #die Dimesnion des Modells Die anzahl der Erwarteten features der inputs/outputs also quasi die anzahl der Wörter in einer sequenz glaube ich -> also 8 bei uns (hier werden die spezial-tokens nicht gezählt ?)
NHEAD = 2 # Anzahl der heads in einem Attention block
FFN_HID_DIM = 512 # Anzahl der hidden layers des Feed-forward networks 
BATCH_SIZE = 16 # wird nicht ans Modell weitergegeben. evtl für uns nicht wichtig, weil wir die Daten schon gebatcht haben?
NUM_ENCODER_LAYERS = 3 # wie viele Encoder blöcke
NUM_DECODER_LAYERS = 3 # wie viele Decoder Blöcke

Jetzt das modell selbst initialisieren und optimizer und Kostenfunktion festlegen

In [143]:
# Modell initialisiern
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
# auf GPU laden
transformer = transformer.to(DEVICE)
# Kostenfunktion als CrossEntropyLoss
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
# Optimizer als Adam optimizer festlegen
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

### 3.2 Trainingsloop definieren

Wichtige Unterschiede Zwischen Guide und Übersetzer Tutorial:

- Definition 1:
    - Führt die Vorhersage mit dem Modell durch und permutiert die Ausgabe, um die Batch-Dimension an die erste Stelle zu setzen (pred = pred.permute(1, 2, 0)).
    - Berechnet den Verlust anhand der permutierten Vorhersage und der erwarteten Ausgabe (loss = loss_fn(pred, y_expected)).

- Definition 2:

    - Führt die Vorhersage mit dem Modell durch, wobei alle Masken übergeben werden (logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)).
    - Berechnet den Verlust direkt aus den Vorhersagen (logits) und der Zielsequenz (tgt_out), indem die Dimensionen der Tensoren für die Verlustfunktion angepasst werden (loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))).

In [144]:
def train_epoch(model, optimizer):
    model.train()
    losses = 0

    for batch in train_dataloader:
        # Source und Target aus dem Dataloader nehem
        src, tgt = batch[:,0], batch[:,1]
        src, tgt = torch.tensor(src).to(DEVICE), torch.tensor(tgt).to(DEVICE)
        
        # tgt eins nach rechts verschieben, sodass mit dem Beginning of stream token der Token auf position 1 predicted wird
        #
        # obacht geben, könnte von den Dimensionen der Daten abhängig sein
        #
        tgt_input = tgt[:,:-1] # die eingehenden daten (Zielsequenz ohne Beginning of stream Token)
        tgt_out = tgt[:,1:] # das gleiche wie y_expected (Zielsequenz ohne End of Stream Token)
        print("tgt_input:", tgt_input)
        print("tgt_out:", tgt_out)
        
        
        # Maske erstellen, um die nächsten wörter zu maskieren
        #
        # Hier genau schauen, was passiert, ich brauche ja nur eine maske eigentlich
        #
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        
        # Verlust wird direkt aus den Logits (Vorhersagen) berechnet
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()
        
        # Hier wahrscheinlich loss berechnen und fehler durchpropagieren -> also training
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))

### 3.3 Validation Loop definieren

In [145]:
def evaluate(model):
    model.eval()
    losses = 0

    

    for batch in val_dataloader:
        # Source und Target aus dem Dataloader nehem
        src, tgt = batch[:,0], batch[:,1]
        src, tgt = torch.tensor(src).to(DEVICE), torch.tensor(tgt).to(DEVICE)
        
        # tgt eins nach rechts verschieben, sodass mit dem Beginning of stream token der Token auf position 1 predicted wird
        #
        # obacht geben, könnte von den Dimensionen der Daten abhängig sein
        #
        tgt_input = tgt[:-1,:] # die eingehenden daten
        tgt_out = tgt[1:,:] # das gleiche wie y_expected
        print("tgt_input:", tgt_input)
        print("tgt_out:", tgt_out)
        

        # Maske erstellen, um die nächsten wörter zu maskieren
        #
        # Hier genau schauen, was passiert, ich brauche ja nur eine maske eigentlich
        #
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        
        # ich weiß nicht genau was hier passiert -> maske, etc wird wahrscheinlich ans modell gegeben
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)


        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

## 4 Modell Trainieren

Hier sind noch nicht die Erwarteten test und validation loss werte. -> maske oder datenverschiebung evtl falsch.

-> erst mal die inferenz abwarten.

In [146]:
from timeit import default_timer as timer
NUM_EPOCHS = 18

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


## 5. Inferenz

In [58]:
# Function for Greedy Decoding
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)
    
    # Memory ist glaube ich einfach die eingabe Encoded
    memory = model.encode(src, src_mask)
    
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        print("target mask:", tgt_mask)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

# Define the prediction function
def predict(model: torch.nn.Module, src):
    model.eval()
    
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model, src, src_mask, max_len=num_tokens + 2, start_symbol=BOS_IDX).flatten()
    
    return tgt_tokens

# Example usage with test sequences
examples = [
    torch.tensor([[2, 0, 0, 0, 0, 0, 0, 0, 0, 3]], dtype=torch.long, device=DEVICE),
    torch.tensor([[2, 1, 1, 1, 1, 1, 1, 1, 1, 3]], dtype=torch.long, device=DEVICE),
    torch.tensor([[2, 1, 0, 1, 0, 1, 0, 1, 0, 3]], dtype=torch.long, device=DEVICE),
    torch.tensor([[2, 0, 1, 0, 1, 0, 1, 0, 1, 3]], dtype=torch.long, device=DEVICE),
    torch.tensor([[2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 3]], dtype=torch.long, device=DEVICE),
    torch.tensor([[2, 0, 1, 3]], dtype=torch.long, device=DEVICE)
]

for idx, example in enumerate(examples):
    result = predict(transformer, example)
    print(f"Example {idx}")
    print(f"Input: {example.view(-1).tolist()[1:-1]}")
    print(f"Continuation: {result[1:-1]}")
    print()
